Add a nice first Markdown cell here

In [7]:
import os
import glob

from tqdm.notebook import tqdm

from utils import extract_text_data_from_pdf

# 1. Identify all PDF files

In [2]:
# Read all files

# Glob all files under ../data/pdfs/

files = glob.glob('../data/pdfs/*')

## 2. Parse PDFs



In [8]:
# TODO: You might want to do this in batches if you have too many files, or it will take a long time.
doc_text = {os.path.basename(file): extract_text_data_from_pdf(file, unstructured_strategy="fast") 
            for file in tqdm(files)}


  0%|          | 0/4 [00:00<?, ?it/s]

Using starter code. Either edit this function or remove this warning if you are happy with the current implementation.
Using starter code. Either edit this function or remove this warning if you are happy with the current implementation.
Using starter code. Either edit this function or remove this warning if you are happy with the current implementation.
Using starter code. Either edit this function or remove this warning if you are happy with the current implementation.


Note that our helpful function already collects a lot of metadata about each chunk of PDF:



In [12]:
list(doc_text.keys())

['cuba_spanish_20250201.pdf',
 'maldives_english_20250201.pdf',
 'zambia_english_20250301.pdf',
 'montenegro_english_20250201.pdf']

In [13]:
doc_text['cuba_spanish_20250201.pdf'][0]

{'id': 0,
 'type': 'Title',
 'text': 'República de Cuba',
 'metadata': {'page_number': 1,
  'coordinates': CoordinatesMetadata(points=((192.127, 238.42984), (192.127, 269.02084), (418.98985600000003, 269.02084), (418.98985600000003, 238.42984)), system=<unstructured.documents.coordinates.PixelSpace object at 0x32d4b8050>),
  'file_directory': '../data/pdfs',
  'filename': 'cuba_spanish_20250201.pdf',
  'languages': ['eng'],
  'last_modified': '2025-03-11T15:38:08',
  'links': [],
  '_known_field_names': frozenset({'attached_to_filename',
             'category_depth',
             'coordinates',
             'data_source',
             'detection_class_prob',
             'detection_origin',
             'emphasized_text_contents',
             'emphasized_text_tags',
             'file_directory',
             'filename',
             'filetype',
             'header_footer_type',
             'image_path',
             'is_continuation',
             'languages',
             'last_m

Now it's up to you...